In [1]:
function fib(n)
    if n<3
        return (result=1,counter=1)
    else
        result1,counter1=fib(n-1)
        result2,counter2=fib(n-2)
        return (result=result1+result2,counter=counter1+counter2+1)
    end
end

@show fib(6)
@show fib(10)
@show fib(20)
#@show fib(50)

fib(6) = (result = 8, counter = 15)
fib(10) = (result = 55, counter = 109)
fib(20) = (result = 6765, counter = 13529)


(result = 6765, counter = 13529)

In [9]:
using Memoize

@memoize fib(n)=n < 3 ? 1 : fib(n-1)+fib(n-2)

@time fib(40)
@time fib(40)
@time fib(50)
@time fib(55)

  0.000020 seconds (180 allocations: 4.531 KiB)
  0.000002 seconds
  0.000014 seconds (51 allocations: 2.906 KiB)
  0.000004 seconds (25 allocations: 400 bytes)


139583862445

In [2]:
using Caching ,CSV,DataFrames

@cache function read_csv(filename::AbstractString)
    println("reading file:",filename)
    @time df=CSV.File(filename) |> DataFrame
    return df
end

df=read_csv("test.csv")
@show df


reading file:test.csv
  0.000611 seconds (687 allocations: 66.805 KiB)
df = 7×24 DataFrame
│ Row │ SaleNumber │ DocketNumber │ ID     │ AttorneyName │ Plaintiff                           │ Defendant                                                                                                                    │ SaleType             │ SaleDate   │ SaleStatus │ PPDate     │ Street                             │ City        │ State  │ ZIPCode │ Address │ Municipality │ CostsTaxes │ ServiceCompleted │ Delivered3129 │ ReadyForSale │ latitude │ longitude │ AdditionalNotes                                                  │ RecordDate │
│     │ String     │ String       │ Int64  │ Missing      │ String                              │ String                                                                                                                       │ String               │ Dates.Date │ String     │ Dates.Da…? │ String                             │ String      │ String │ Int64   │ Miss

,SaleNumber,DocketNumber,ID,AttorneyName,Plaintiff
,String,String,Int64,Missing,String
1,004SEP20,GD-19-009369,656672,missing,"""Elizabeth Forward School District"""
2,007SEP20,GD-20-006099,670923,missing,"""1510 Fifth Avenue LLC"""
3,005SEP20,GD-19-010786,657780,missing,"""Elizabeth Forward School District"""
4,001SEP20,GD-20-006557,670161,missing,"""SSB Bank"""
5,002SEP20,GD-18-010328,642572,missing,"""Penn Hills School District"""
6,003SEP20,GD-19-006119,653844,missing,"""South Allegheny School District"""
7,006SEP20,GD-17-010126,624849,missing,"""Elizabeth Forward School District"""


In [3]:
@show size(df)
@show read_csv.cache

size(df) = (7, 24)
read_csv.cache = Dict{UInt64,Any}(0x58fc842981b483ea => 7×24 DataFrame
│ Row │ SaleNumber │ DocketNumber │ ID     │ AttorneyName │ Plaintiff                           │ Defendant                                                                                                                    │ SaleType             │ SaleDate   │ SaleStatus │ PPDate     │ Street                             │ City        │ State  │ ZIPCode │ Address │ Municipality │ CostsTaxes │ ServiceCompleted │ Delivered3129 │ ReadyForSale │ latitude │ longitude │ AdditionalNotes                                                  │ RecordDate │
│     │ String     │ String       │ Int64  │ Missing      │ String                              │ String                                                                                                                       │ String               │ Dates.Date │ String     │ Dates.Da…? │ String                             │ String      │ String │ Int64   │ Missi

Dict{UInt64,Any} with 1 entry:
  0x58fc842981b483ea => 7×24 DataFrame. Omitted printing of 20 columns…

In [9]:
random_data(n)=isodd(n) ? rand(Int,n) : rand(Float64,n)

@code_warntype random_data(10)

Variables
  #self#::Core.Compiler.Const(random_data, false)
  n::Int64

Body::Union{Array{Float64,1}, Array{Int64,1}}
1 ─ %1 = Main.isodd(n)::Bool
└──      goto #3 if not %1
2 ─ %3 = Main.rand(Main.Int, n)::Array{Int64,1}
└──      return %3
3 ─ %5 = Main.rand(Main.Float64, n)::Array{Float64,1}
└──      return %5


In [13]:
using BenchmarkTools
function double_sum_of_random_data(n)
    data=random_data(n)
    total=0
    for v  in data
        total+z=2*v
    end
    return total
end
@btime double_sum_of_random_data(100000)
@btime double_sum_of_random_data(100001)

  10.963 ms (200002 allocations: 3.81 MiB)
  12.472 ms (200004 allocations: 3.81 MiB)


0

In [15]:
@code_warntype double_sum_of_random_data(1000)

Variables
  #self#::Core.Compiler.Const(double_sum_of_random_data, false)
  n::Int64
  data::Union{Array{Float64,1}, Array{Int64,1}}
  total::Int64
  @_5::Union{Nothing, Tuple{Union{Float64, Int64},Int64}}
  v::Union{Float64, Int64}
  +::var"#+#12"{_A} where _A

Body::Int64
1 ─       (data = Main.random_data(n))
│         (total = 0)
│   %3  = data::Union{Array{Float64,1}, Array{Int64,1}}
│         (@_5 = Base.iterate(%3))
│   %5  = (@_5 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #4 if not %6
2 ┄ %8  = @_5::Tuple{Union{Float64, Int64},Int64}::Tuple{Union{Float64, Int64},Int64}
│         (v = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│   %11 = Main.:(var"#+#12")::Core.Compiler.Const(var"#+#12", false)
│   %12 = Core.typeof(v)::Union{Type{Float64}, Type{Int64}}
│   %13 = Core.apply_type(%11, %12)::Type{var"#+#12"{_A}} where _A
│         (+ = %new(%13, v))
│         (@_5 = Base.iterate(%3, %10))
│   %16 = (@_5 === nothing)::Bool
│   %17 = Base.not

In [20]:
function double_sum(data)
    total=0
    for v  in data
        total+=2v
    end
    return total
end

function  double_sum_of_random_data(n)
    data=random_data(n)
    return double_sum(data)
end

@btime double_sum_of_random_data(10001)


  13.800 μs (2 allocations: 78.27 KiB)


1563301388332697656

In [25]:
@code_warntype double_sum(rand(Int,10))
println("##############################\n")
@code_warntype double_sum(rand(Float64,10))

Variables
  #self#::Core.Compiler.Const(double_sum, false)
  data::Array{Int64,1}
  total::Int64
  @_4::Union{Nothing, Tuple{Int64,Int64}}
  v::Int64

Body::Int64
1 ─       (total = 0)
│   %2  = data::Array{Int64,1}
│         (@_4 = Base.iterate(%2))
│   %4  = (@_4 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_4::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (v = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = total::Int64
│   %11 = (2 * v)::Int64
│         (total = %10 + %11)
│         (@_4 = Base.iterate(%2, %9))
│   %14 = (@_4 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return total
##############################

Variables
  #self#::Core.Compiler.Const(double_sum, false)
  data::Array{Float64,1}
  total::Union{Float64, Int64}
  @_4::Union{Nothing, Tuple{Float64,Int64}}
  v::Float64

Body::Union{Float64, Int64}
1 ─       (total = 0)
│   %2  = data:

In [26]:
function double_sum(datat::AbstractVector{T}) where {T <: Number}
    total=zero(T)
    for v in data
        total+=v
    end
    return total
end



double_sum (generic function with 2 methods)

In [28]:
using Test
@inferred random_data(1)

LoadError: return type Array{Int64,1} does not match inferred return type Union{Array{Float64,1}, Array{Int64,1}}